<a href="https://colab.research.google.com/github/farhanarnob/BioCounting/blob/main/GreedyMotifSearch%20with%20Pseudocounts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import sys
import numpy as np 
nucleotides = ["A","C","G","T"]
def hammingDistance(stringOne, stringTwo):
    count = 0
    for i in range(0,len(stringOne)):
        if stringOne[i]!=stringTwo[i]:
            count += 1
    return count

def generator(s, k):
    for i in range(1 + len(s) - k): yield s[i:i+k]

def calProfile(profile, kmer):
    probability = 1.0
    for i in range (len(kmer)):
        if kmer[i] == "A":
            probability = probability * float(profile[i][0])
        elif kmer[i] == "C":
            probability = probability * float(profile[i][1])
        elif kmer[i] == "G":
            probability = probability * float(profile[i][2])
        elif kmer[i] == "T":
            probability = probability * float(profile[i][3])
    return probability

def mostProbableProfile(text,k,profile):
    maxPrblity = 0.0
    res = ""
    for i in range(len(text)- k + 1):
        kmer = text[i:i+k]
        prblity = calProfile(profile, kmer)
        if prblity > maxPrblity:
            maxPrblity = prblity
            res = kmer
    return res 

def count(Motifs):
    count = {}
    k = len(Motifs[0])
    for symbol in "ACGT":
        count[symbol] = []
        for i in range(k):
            count[symbol].append(0)
    t = len(Motifs)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

def findConsensus(motifs):

    consensus = ""
    for i in range(len(motifs[0])):
        countA, countC, countG, countT = 0, 0, 0, 0
        for motif in motifs:
            if motif[i] == nucleotides[0]:
                countA += 1
            elif motif[i] == nucleotides[1]:
                countC += 1
            elif motif[i] == nucleotides[2]:
                countG += 1
            elif motif[i] == nucleotides[3]:
                countT += 1
        if countA >= max(countC, countG, countT):
            consensus += nucleotides[0]
        elif countC >= max(countA, countG, countT):
            consensus += nucleotides[1]
        elif countG >= max(countC, countA, countT):
            consensus += nucleotides[2]
        elif countT >= max(countC, countG, countA):
            consensus += nucleotides[3]
    return consensus


def profileMatrix(motifs):
    Profile = {}
    A, C, G, T = [], [], [], []
    for j in range(len(motifs[0])):
        countA, countC, countG, countT = 0, 0, 0, 0
        for motif in motifs:
            if motif[j] == nucleotides[0]:
                countA += 1
            elif motif[j] == nucleotides[1]:
                countC += 1
            elif motif[j] == nucleotides[2]:
                countG += 1
            elif motif[j] == nucleotides[3]:
                countT += 1
        A.append(countA)
        C.append(countC)
        G.append(countG)
        T.append(countT)
    Profile["A"] = A
    Profile["C"] = C
    Profile["G"] = G
    Profile["T"] = T
    return Profile


def score(motifs):
    consensus = findConsensus(motifs)
    score = 0.0000
    for motif in motifs:
        score += hammingDistance(consensus, motif)
    return round(score, 4)

def greedyMotifSearch(DNA, k, t):
    import math
    bestMotifs = []
    bestScore = math.inf
    for string in DNA:
        bestMotifs.append(string[:k])
    base = DNA[0]
    for i in generator(base, k):
       
        newMotifs = [i]
        
        for j in range(1, len(DNA)):
           
            profile = profileMatrix(newMotifs)
            probable = mostProbableProfile(DNA[j], k, profile)
            newMotifs.append(probable)

        if score(newMotifs) < bestScore:
       
            bestScore = score(newMotifs)
            bestMotifs = newMotifs
    return bestMotifs



def profilePseudocounts(motifs):
	'''Returns the profile of the dna list motifs.'''
	prof = []
	for i in range(len(motifs[0])):
		col = ''.join([motifs[j][i] for j in range(len(motifs))])
		prof.append([float(col.count(nuc)+1)/float(len(col)+4) for nuc in 'ACGT'])
	return prof

def greedyMotifSearchAlgorithm(Dna, k, t):
    # Initialize the best score as a score higher than the highest possible score.
    bestScore = k*t

    # Run the greedy motif search.
    for i in range(len(Dna[0])-k+1):
        # Initialize the motifs as each k-mer from the first dna sequence.
        motifs = [Dna[0][i:i+k]]

        # Find the most probable k-mer in the next string.
        for j in range(1, t):
            currentProfile = profilePseudocounts(motifs)
            motifs.append(mostProbableProfile(Dna[j], k, currentProfile))

        # Check to see if we have a new best scoring list of motifs.
        currentScore = score(motifs)
        if currentScore < bestScore:
            bestScore = currentScore
            bestMotifs = motifs

    return bestMotifs
def main():
    # reads input from terminal
    input = "".join(open('/content/rosalind_ba2e.txt')).split()
    # read input
    k = int(input[0])
    t = int(input[1])
    dna = input[2:]

    bestMotifs = greedyMotifSearchAlgorithm(dna, k, t)
    print(*bestMotifs)


if __name__ == "__main__":
    main()

GCTTACGTATAC GCTTATGGAGGC GCTTATGAAACC GCTTAAGCATTC GCTTAAGAAACC GCTTACGAACTC GCTTATGAAAAC GCTTAGGTATCC GCTTAGGGAGTC GCTTAAGGAACC GCTTAAGGACTC GCTTAAGGAATC GCTTATGCATAC GCTTACGGATTC GCTTAAGTAGGC GCTTAAGCAGCC GCTTATGGAGAC GCTTACGGACAC GCTTATGAACGC GCTTAAGAAATC GCTTACGTAATC GCTTACGTAACC GCTTAGGAACAC GCTTAAGAAAGC GCTTAAGTAAGC
